<a href="https://colab.research.google.com/github/sqbitegh/Colabs/blob/main/phi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


def init_model():
    model_name = "microsoft/phi-2" # Replace with the actual model name on Hugging Face Hub
    model = AutoModelForCausalLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return model, tokenizer

def init_tinyllama_model():
    model_name = "TinyLlama/TinyLlama_v1.1_math_code"
    model = AutoModelForCausalLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return model, tokenizer

In [4]:
import os
import numpy as np

def save_layer_activations(hidden_states, layer_index, output_dir="layer_activations", filename="activations.txt"):
    """
    Saves the hidden states of a specific layer to a text file.

    Args:
        hidden_states (tuple): Tuple of hidden states from all layers.
        layer_index (int): The index of the layer to save activations from.
        output_dir (str): The directory to save the output file.
        filename (str): The name of the output file.
    """
    if layer_index < len(hidden_states):
        layer_activations = hidden_states[layer_index]

        # Assuming batch size is 1 and sequence length is the second dimension
        # The shape is typically (batch_size, sequence_length, hidden_size)
        # We need to save activations for each token in the sequence

        # Create the output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)

        file_path = os.path.join(output_dir, filename)

        # Open in append mode to write activations from different inference calls
        with open(file_path, 'a') as f:
            # Iterate through each token in the sequence
            # Assuming batch size is 1, we access layer_activations[0]
            for token_activations in layer_activations[0].cpu().detach().numpy():
                np.savetxt(f, token_activations.reshape(1, -1), fmt='%f') # Use numpy.savetxt, reshape for 1D array
                f.write('\n') # Add a newline after each token's activations

        print(f"Activations from layer {layer_index} saved to {file_path}")
    else:
        print(f"Error: Layer index {layer_index} is out of bounds.")




def run_inference(model, tokenizer, input_prompt, activations_filename):
    input_text = input_prompt
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    print(f"tokenizer.eos_token_id : {tokenizer.eos_token_id}")
    # Configure the model to return hidden states
    with torch.no_grad(): # Disable gradient calculation for inference
        outputs = model(input_ids, output_hidden_states=False)
        generated_ids = model.generate(input_ids, max_length=150) # You can adjust max_length

    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    print("\nGenerated text:")
    print(generated_text)



    whole_text = input_text + generated_text
    whole_text_ids = tokenizer.encode(whole_text, return_tensors="pt")

    print("whole_text_ids & tokens:")
    token_ids_list = whole_text_ids[0].tolist()

    print("Index | Token ID | Decoded Token")
    print("------|----------|--------------")

    for index, token_id in enumerate(token_ids_list):
        decoded_token = tokenizer.decode(token_id)
        print(f"{index:<5} | {token_id:<8} | {decoded_token}")


    with torch.no_grad(): # Disable gradient calculation for inference
        wholetext_outputs = model(whole_text_ids, output_hidden_states=True)
    print(f"whole_text_ids {whole_text_ids} {whole_text_ids.shape}")

    # Access the hidden states
    # outputs.hidden_states is a tuple where each element is the hidden state for a layer
    # The first element is the embedding layer output, and the last is the output before the classification head
    hidden_states = wholetext_outputs.hidden_states

    print(f"  Layer {32}: {hidden_states[32].shape}")
    #print("Shapes of hidden states for all layers:")

    #for i, layer_hidden_state in enumerate(hidden_states):
    #    print(f"  Layer {i}: {layer_hidden_state.shape}")
    #    #print(f"  Layer {i}: {layer_hidden_state}")

    save_layer_activations(hidden_states, 32, "layer_activations", activations_filename)





In [ ]:
model, tokenizer = init_model()

model, tokenizer = init_tinyllama_model()


In [4]:
print(model.config)

PhiConfig {
  "architectures": [
    "PhiForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 50256,
  "embd_pdrop": 0.0,
  "eos_token_id": 50256,
  "hidden_act": "gelu_new",
  "hidden_size": 2560,
  "initializer_range": 0.02,
  "intermediate_size": 10240,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "phi",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "partial_rotary_factor": 0.4,
  "qk_layernorm": false,
  "resid_pdrop": 0.1,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.52.2",
  "use_cache": true,
  "vocab_size": 51200
}



In [ ]:
#run_inference(model, tokenizer, "Your input text here")

run_inference(model, tokenizer, "example factorial function in c: ", "activations_phi2_factor_c3.txt")